# Retail Sales (Istanbul malls). Revenue predict with Random Forest Regressor

https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset

## Import libraries

In [1]:
import pandas as pd
import warnings
import keyring
import requests
from dateutil.relativedelta import relativedelta
import os
import numpy as np
import datetime
from sklearn import ensemble
import matplotlib.pyplot as plt
from datetime import date, timedelta
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, accuracy_score, precision_score, f1_score, roc_auc_score, mean_squared_error
import seaborn as sns

## Load data

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv('/workspaces/dbt_ml_retail/Forecast/customer_shopping_data.csv')

https://drive.google.com/file/d/1syhtZtr0n57iHuc2MYDrorM_ihu5AqRz/view?usp=drive_link

In [4]:
!gdown 1syhtZtr0n57iHuc2MYDrorM_ihu5AqRz #Other way to download from gdrive

Downloading...
From: https://drive.google.com/uc?id=1syhtZtr0n57iHuc2MYDrorM_ihu5AqRz
To: /content/customer_shopping_data.csv
100% 7.54M/7.54M [00:00<00:00, 44.4MB/s]


In [5]:
df = pd.read_csv('/content/customer_shopping_data.csv')

In [6]:
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.400,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.510,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.080,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.850,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.600,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.650,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.460,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.460,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.000,Cash,16/03/2021,Istinye Park


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


## Key metric - Revenue

In [7]:
df['revenue'] = df['quantity']*df['price']

In [8]:
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,revenue
0,I138884,C241288,Female,28,Clothing,5,1500.400,Credit Card,5/8/2022,Kanyon,7502.000
1,I317333,C111565,Male,21,Shoes,3,1800.510,Debit Card,12/12/2021,Forum Istanbul,5401.530
2,I127801,C266599,Male,20,Clothing,1,300.080,Cash,9/11/2021,Metrocity,300.080
3,I173702,C988172,Female,66,Shoes,5,3000.850,Credit Card,16/05/2021,Metropol AVM,15004.250
4,I337046,C189076,Female,53,Books,4,60.600,Cash,24/10/2021,Kanyon,242.400
...,...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.650,Credit Card,21/09/2022,Kanyon,293.250
99453,I325143,C569580,Male,27,Food & Beverage,2,10.460,Cash,22/09/2021,Forum Istanbul,20.920
99454,I824010,C103292,Male,63,Food & Beverage,2,10.460,Debit Card,28/03/2021,Metrocity,20.920
99455,I702964,C800631,Male,56,Technology,4,4200.000,Cash,16/03/2021,Istinye Park,16800.000


## EDA

In [11]:
def highlight_max(s):
    is_max = s == s.max()
    return ['color: green' if cell else '' for cell in is_max]

In [ ]:
df['invoice_date'].max()

'9/9/2022'

In [ ]:
df['invoice_date'].min()

'1/1/2021'

In [ ]:
df_gender = df.groupby('gender')['revenue'].agg(['count','mean','sum'])
df_gender[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
gender,,,
Female,59482,2525.253623,150207136.020000
Male,39975,2534.050237,101298658.230000



Women predominate in the frame

In [ ]:
df_payment_method = df.groupby('payment_method')['revenue'].agg(['count','mean','sum'])
df_payment_method[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
payment_method,,,
Cash,44447,2538.579500,112832243.020000
Credit Card,34931,2521.460129,88077123.770000
Debit Card,20079,2519.867895,50596427.460000


Cash is used more for purchases

In [ ]:
df_category = df.groupby('category')['revenue'].agg(['count','mean','sum'])
df_category[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
category,,,
Books,4981,167.547260,834552.900000
Clothing,34487,3305.500364,113996791.040000
Cosmetics,15097,449.947864,6792862.900000
Food & Beverage,14776,57.494251,849535.050000
Shoes,10034,6632.793649,66553451.470000
Souvenir,4999,127.190368,635824.650000
Technology,4996,11581.735388,57862350.000000
Toys,10087,394.609521,3980426.240000


Clothing, shoes & technology are most popular categories per revenue

In [ ]:
df['age'].min()

18

In [ ]:
df['age'].max()

69

Bins of age group:

In [9]:
df['age_group'] = pd.cut(df['age'],[15,20,25,30,35,40,45,50,55,60,65,np.inf])

In [12]:
df_age_group = df.groupby('age_group')['revenue'].agg(['count','mean','sum'])
df_age_group[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
age_group,,,
"(15.0, 20.0]",5624,2428.677585,13658882.740000
"(20.0, 25.0]",9735,2512.520658,24459388.610000
"(25.0, 30.0]",9665,2579.320306,24929130.760000
"(30.0, 35.0]",9394,2443.105048,22950528.820000
"(35.0, 40.0]",9872,2574.947782,25419884.500000
"(40.0, 45.0]",9564,2589.329913,24764351.290000
"(45.0, 50.0]",9502,2527.414785,24015495.290000
"(50.0, 55.0]",9514,2544.066360,24204247.350000
"(55.0, 60.0]",9418,2504.185096,23584415.230000


* Age group between 15 and 20/ 65+ is the least active.
* The sample is balanced by age.
* Age group between 35 and 45 is the aim for us.

Load base data with correct date format:

In [13]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format='%d/%m/%Y')
df.to_csv('df_base.csv')

In [15]:
df_shopping_mall = df.groupby('shopping_mall')['revenue'].agg(['count','mean','sum'])
df_shopping_mall[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
shopping_mall,,,
Cevahir AVM,4991,2533.588099,12645138.200000
Emaar Square Mall,4811,2578.694718,12406100.290000
Forum Istanbul,4947,2487.148017,12303921.240000
Istinye Park,9781,2517.005181,24618827.680000
Kanyon,19823,2550.281547,50554231.100000
Mall of Istanbul,19943,2550.894132,50872481.680000
Metrocity,15011,2485.030133,37302787.330000
Metropol AVM,10161,2497.777108,25379913.190000
Viaport Outlet,4914,2548.095181,12521339.720000


The most popular mall is Mall of Istanbul

## Feature engineering

Lets create features for our model

In [16]:
start_forecast = "2022-09-10"
end_forecast = "2023-12-31"

In [17]:
df.columns

Index(['invoice_no', 'customer_id', 'gender', 'age', 'category', 'quantity',
       'price', 'payment_method', 'invoice_date', 'shopping_mall', 'revenue',
       'age_group'],
      dtype='object')

In [18]:
df_model = df[['gender'#,'age'
,'age_group','category','payment_method','invoice_date','shopping_mall','revenue']]
df_model.head()

,gender,age_group,category,payment_method,invoice_date,shopping_mall,revenue
0,Female,"(25.0, 30.0]",Clothing,Credit Card,2022-08-05,Kanyon,7502.000
1,Male,"(20.0, 25.0]",Shoes,Debit Card,2021-12-12,Forum Istanbul,5401.530
2,Male,"(15.0, 20.0]",Clothing,Cash,2021-11-09,Metrocity,300.080
3,Female,"(65.0, inf]",Shoes,Credit Card,2021-05-16,Metropol AVM,15004.250
4,Female,"(50.0, 55.0]",Books,Cash,2021-10-24,Kanyon,242.400


In [19]:
df_model['invoice_date'] = pd.to_datetime(df_model['invoice_date'], format='%d/%m/%Y')

In [20]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gender          99457 non-null  object        
 1   age_group       99457 non-null  category      
 2   category        99457 non-null  object        
 3   payment_method  99457 non-null  object        
 4   invoice_date    99457 non-null  datetime64[ns]
 5   shopping_mall   99457 non-null  object        
 6   revenue         99457 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(4)
memory usage: 4.6+ MB


In [ ]:
df_model.head()

,gender,age_group,category,payment_method,invoice_date,shopping_mall,revenue
0,Female,"(25.0, 30.0]",Clothing,Credit Card,2022-08-05,Kanyon,7502.000
1,Male,"(20.0, 25.0]",Shoes,Debit Card,2021-12-12,Forum Istanbul,5401.530
2,Male,"(15.0, 20.0]",Clothing,Cash,2021-11-09,Metrocity,300.080
3,Female,"(65.0, inf]",Shoes,Credit Card,2021-05-16,Metropol AVM,15004.250
4,Female,"(50.0, 55.0]",Books,Cash,2021-10-24,Kanyon,242.400


In [ ]:
df_model.sort_values(by='invoice_date',ascending=True)

,gender,age_group,category,payment_method,invoice_date,shopping_mall,revenue
36199,Female,"(65.0, inf]",Food & Beverage,Credit Card,2021-01-01,Metrocity,130.750
78539,Female,"(40.0, 45.0]",Clothing,Credit Card,2021-01-01,Mall of Istanbul,4801.280
27812,Male,"(50.0, 55.0]",Clothing,Cash,2021-01-01,Metrocity,2700.720
27856,Male,"(55.0, 60.0]",Food & Beverage,Credit Card,2021-01-01,Mall of Istanbul,83.680
5323,Male,"(30.0, 35.0]",Clothing,Cash,2021-01-01,Istinye Park,300.080
...,...,...,...,...,...,...,...
20863,Female,"(65.0, inf]",Clothing,Cash,2023-03-08,Mall of Istanbul,7502.000
20879,Female,"(35.0, 40.0]",Food & Beverage,Cash,2023-03-08,Kanyon,47.070
61644,Female,"(55.0, 60.0]",Cosmetics,Credit Card,2023-03-08,Kanyon,1016.500
27871,Female,"(45.0, 50.0]",Shoes,Credit Card,2023-03-08,Metrocity,5401.530


In [ ]:
df_model.columns

Index(['gender', 'age_group', 'category', 'payment_method', 'invoice_date',
       'shopping_mall', 'revenue'],
      dtype='object')

In [ ]:
gender = pd.DataFrame({"gender": df_model.gender.drop_duplicates()})
age_group = pd.DataFrame({"age_group": df_model.age_group.drop_duplicates()})
category = pd.DataFrame({"category": df_model.category.drop_duplicates()})
payment_method = pd.DataFrame({"payment_method": df_model.payment_method.drop_duplicates()})
shopping_mall = pd.DataFrame({"shopping_mall": df_model.shopping_mall.drop_duplicates()})
dates = pd.DataFrame({"invoice_date": pd.date_range(start=min(df_model.invoice_date), end=end_forecast, freq="D")})

gender["key"], age_group ["key"], dates["key"], category["key"], payment_method["key"], shopping_mall["key"] = 0, 0, 0, 0, 0, 0

df_all = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(gender, age_group , on="key"), dates, on="key"), category, on="key"), payment_method, on="key"), shopping_mall, on="key")
df_all = df_all.drop("key", axis=1)

df = pd.merge(df_all, df_model, on=["gender", "age_group", "category", "payment_method", "shopping_mall", "invoice_date"], how="left")
df["revenue"] = np.where(df["revenue"].isnull(), 0, df["revenue"])

df["year"] = df["invoice_date"].dt.year
df["day"] = df['invoice_date'].dt.day
df["month"] = df["invoice_date"].dt.month
df["weekday"] = df["invoice_date"].dt.weekday

df["is1jan"] = np.where((df["month"] == 1) & (df["invoice_date"].dt.day == 1), 1, 0)
df["wn"] = np.where(df["weekday"] <= 4, 1, 0)
df

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,is1jan,wn
0,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Kanyon,0.000,2021,1,1,4,1,1
1,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Forum Istanbul,0.000,2021,1,1,4,1,1
2,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metrocity,0.000,2021,1,1,4,1,1
3,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metropol AVM,0.000,2021,1,1,4,1,1
4,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Istinye Park,0.000,2021,1,1,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5784352,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Mall of Istanbul,0.000,2023,31,12,6,0,0
5784353,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Emaar Square Mall,0.000,2023,31,12,6,0,0
5784354,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Cevahir AVM,0.000,2023,31,12,6,0,0
5784355,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Viaport Outlet,0.000,2023,31,12,6,0,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
df['gender_ohe']=le.fit_transform(df['gender']).astype('str')
df['category_ohe']=le.fit_transform(df['category']).astype('str')
df['payment_method_ohe']=le.fit_transform(df['payment_method']).astype('str')
df['shopping_mall_ohe']=le.fit_transform(df['shopping_mall']).astype('str')
df['age_group_ohe']=le.fit_transform(df['age_group']).astype('str')

In [ ]:
df.head()

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,is1jan,wn,gender_ohe,category_ohe,payment_method_ohe,shopping_mall_ohe,age_group_ohe
0,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Kanyon,0.000,2021,1,1,4,1,1,0,1,1,4,2
1,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Forum Istanbul,0.000,2021,1,1,4,1,1,0,1,1,2,2
2,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metrocity,0.000,2021,1,1,4,1,1,0,1,1,6,2
3,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metropol AVM,0.000,2021,1,1,4,1,1,0,1,1,7,2
4,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Istinye Park,0.000,2021,1,1,4,1,1,0,1,1,3,2


In [ ]:
features_base_model = ["gender_ohe", "age_group_ohe", "category_ohe","payment_method_ohe","shopping_mall_ohe", "day","month","weekday","is1jan","wn"]

## Revenue forecast

In [ ]:
reg = ensemble.RandomForestRegressor(random_state =0,n_estimators=10)

In [ ]:
reg.fit(df[features_base_model], df["revenue"])

RandomForestRegressor(n_estimators=10, random_state=0)

In [ ]:
df["predict_revenue"] = reg.predict(df[features_base_model])

## Evaluate of error

In [ ]:
def mean_absolute_error_day(y):
    y_piv = y.groupby(['invoice_date'])['predict_revenue', 'revenue'].sum().reset_index()
    y_piv['mae_revenue'] = np.abs(y_piv['predict_revenue'] - y_piv['revenue'])
    return np.mean(y_piv['mae_revenue'])/np.mean(y_piv['revenue'])

In [ ]:
df_test = df[(df["invoice_date"] < start_forecast ) ]

In [ ]:
mean_absolute_error_day(df_test)

0.12312527027243632

Error - 12%

In [ ]:
r2_score(df['revenue'], df['predict_revenue'])

0.7487469811440195

R-2 score is 75% which indicated as a high level of correlation


## Lets create the final dataset with predicted values

In [ ]:
df_predict = (df[['gender','age_group','invoice_date'
                ,'category','payment_method','shopping_mall','predict_revenue']]
                 [(df["invoice_date"] >= start_forecast )
                 & (df["invoice_date"] <= end_forecast)
                 & (df['predict_revenue'] != 0) ])

df_predict

,gender,age_group,invoice_date,category,payment_method,shopping_mall,predict_revenue
148237,Female,"(25.0, 30.0]",2022-09-10,Clothing,Credit Card,Kanyon,3360.896
148239,Female,"(25.0, 30.0]",2022-09-10,Clothing,Credit Card,Metrocity,3751.000
148242,Female,"(25.0, 30.0]",2022-09-10,Clothing,Credit Card,Mall of Istanbul,1230.328
148249,Female,"(25.0, 30.0]",2022-09-10,Clothing,Debit Card,Metrocity,960.256
148259,Female,"(25.0, 30.0]",2022-09-10,Clothing,Cash,Metrocity,270.072
...,...,...,...,...,...,...,...
5784228,Male,"(40.0, 45.0]",2023-12-31,Cosmetics,Cash,Forum Istanbul,4.066
5784241,Male,"(40.0, 45.0]",2023-12-31,Food & Beverage,Credit Card,Istinye Park,2.092
5784252,Male,"(40.0, 45.0]",2023-12-31,Food & Beverage,Debit Card,Mall of Istanbul,0.523
5784264,Male,"(40.0, 45.0]",2023-12-31,Food & Beverage,Cash,Cevahir AVM,13.075


In [ ]:
df_predict.to_csv('df_predict.csv')